In [1]:
import numpy as np
import pandas as pd

In [2]:
from pymatreader import read_mat

data = read_mat('../data/raw/1st_Day.mat')

In [3]:
data.keys()

dict_keys(['subs_ica'])

In [4]:
len(data['subs_ica'])

15

In [5]:
type(data['subs_ica'][0])

dict

In [6]:
data['subs_ica'][0].keys()

dict_keys(['left_im2', 'right_quasi', 'left_quasi', 'left_im1', 'left_real', 'right_real', 'right_im1', 'right_im2'])

In [7]:
type(data['subs_ica'][0]['right_real'])

dict

In [8]:
data['subs_ica'][0]['right_real'].keys()

dict_keys(['label', 'time', 'fsample', 'cfg', 'trial', 'elec', 'hdr', 'sampleinfo', 'trialinfo'])

In [10]:
data['subs_ica'][0]['right_real']['trial'][0].shape

(32, 10012)

In [11]:
for trial in data['subs_ica'][0]['right_real']['trial']:
    print(trial.shape)

(32, 10012)
(32, 9975)
(32, 9975)
(32, 9976)
(32, 9965)
(32, 9977)
(32, 9973)
(32, 9975)
(32, 9975)
(32, 9964)
(32, 9975)
(32, 9975)
(32, 9975)
(32, 9974)
(32, 9978)
(32, 9976)
(32, 9976)
(32, 9974)
(32, 9975)
(32, 9968)


In [13]:
max_length = 0
for person_idx, person in enumerate(data['subs_ica']):
    for data_type in person.keys():
        times = person[data_type]['time']
        max_len = max([time.shape[0] for time in times])
        max_length = max(max_length, max_len)
max_length

10095

In [11]:
for key, value in data['subs_ica'][0]['right_real'].items():
    print(f'{key} - {type(value)}')
    if isinstance(value, list):
        print(f'\t{len(value)} - {value[:10]}')
    elif isinstance(value, np.ndarray):
        print(f'\t{value.shape} - {value[:10]}')
    elif isinstance(value, dict):
        print(f'\t{value.keys()}')

cfg - <class 'dict'>
	dict_keys(['demean', 'component', 'trackmeminfo', 'previous', 'tracktimeinfo', 'updatesens', 'progress', 'callinfo', 'toolbox', 'feedback', 'version', 'checkpath', 'outputfilepresent'])
trial - <class 'list'>
	20 - [array([[-6.72739269e-06, -1.65505850e-06,  3.33864236e-06, ...,
         3.01690979e-05,  2.76408663e-05,  2.48525767e-05],
       [ 1.36194253e-05,  6.88946381e-06,  9.70009839e-08, ...,
         2.15157432e-05,  1.84133675e-05,  1.44326846e-05],
       [-4.17162079e-06, -6.12514510e-06, -7.83398611e-06, ...,
        -2.20993728e-07,  7.46996658e-07,  2.10515599e-06],
       ...,
       [ 7.31326999e-06,  8.99689259e-06,  1.10115116e-05, ...,
        -2.61372556e-05, -2.01016820e-05, -1.20095963e-05],
       [ 1.34758667e-05,  1.36613811e-05,  1.41711303e-05, ...,
        -2.02139549e-05, -1.64671561e-05, -1.15712975e-05],
       [ 1.70988841e-05,  1.50062545e-05,  1.32532145e-05, ...,
        -1.83361715e-05, -1.52687593e-05, -1.14006257e-05]]), arra

In [12]:
right_real_times = []
right_real_trials = []
person_idxs = []
trial_idxs = []
sensor_idxs = []

for person_idx, person in enumerate(data['subs_ica']):
    times = person['right_real']['time']
    trials = person['right_real']['trial']

    for trial_idx, (time, trial) in enumerate(zip(times, trials)):
        for sensor_idx, sensor_data in enumerate(trial):
            right_real_times.append(time)
            right_real_trials.append(sensor_data)
            person_idxs.append(person_idx)
            trial_idxs.append(trial_idx)
            sensor_idxs.append(sensor_idx)

In [13]:
len(right_real_times), len(right_real_times[0])

(9152, 10012)

In [14]:
for time in right_real_times:
    if len(time) != len(right_real_times[0]):
        print(f'len = {len(time)}')
        break

len = 9975


In [15]:
min_len, max_len = min([len(time) for time in right_real_times]), max([len(time) for time in right_real_times])
min_len, max_len

(9960, 10095)

In [16]:
time_array = np.full((len(right_real_times), max_len), np.nan)
trial_array = np.full((len(right_real_times), max_len), np.nan)

for idx, (time, trial) in enumerate(zip(right_real_times, right_real_trials)):
    actual_len = len(time)
    time_array[idx, :actual_len] = time
    trial_array[idx, :actual_len] = trial

In [17]:
time_array.shape, trial_array.shape

((9152, 10095), (9152, 10095))

In [18]:
time_array[:5]

array([[0.   , 0.001, 0.002, ...,   nan,   nan,   nan],
       [0.   , 0.001, 0.002, ...,   nan,   nan,   nan],
       [0.   , 0.001, 0.002, ...,   nan,   nan,   nan],
       [0.   , 0.001, 0.002, ...,   nan,   nan,   nan],
       [0.   , 0.001, 0.002, ...,   nan,   nan,   nan]])

In [19]:
right_real_info = pd.DataFrame({
    'person_idx': np.array(person_idxs),
    'trial_idx': np.array(trial_idxs),
    'sensor_idx': np.array(sensor_idxs),
})
right_real_info.head(10)

,person_idx,trial_idx,sensor_idx
0,0,0,0
1,0,0,1
2,0,0,2
3,0,0,3
4,0,0,4
5,0,0,5
6,0,0,6
7,0,0,7
8,0,0,8
9,0,0,9


In [20]:
right_real_info.to_parquet('../data/right_real_info.parquet')

In [21]:
np.save('../data/right_real_time.npy', time_array)
np.save('../data/right_real_trial.npy', trial_array)

In [28]:
def extract_data(data: dict, data_type: str, save: bool = False):
    result_times = []
    result_trials = []
    person_idxs = []
    trial_idxs = []
    sensor_idxs = []

    for person_idx, person in enumerate(data['subs_ica']):
        times = person[data_type]['time']
        trials = person[data_type]['trial']

        for trial_idx, (time, trial) in enumerate(zip(times, trials)):
            for sensor_idx, sensor_data in enumerate(trial):
                result_times.append(time)
                result_trials.append(sensor_data)
                person_idxs.append(person_idx)
                trial_idxs.append(trial_idx)
                sensor_idxs.append(sensor_idx)

    max_len = max([len(time) for time in result_times])

    time_array = np.full((len(result_times), max_len), np.nan)
    trial_array = np.full((len(result_trials), max_len), np.nan)

    for idx, (time, trial) in enumerate(zip(result_times, result_trials)):
        actual_len = len(time)
        time_array[idx, :actual_len] = time
        trial_array[idx, :actual_len] = trial

    info = pd.DataFrame({
        'person_idx': np.array(person_idxs),
        'trial_idx': np.array(trial_idxs),
        'sensor_idx': np.array(sensor_idxs),
    })

    if save:
        info.to_parquet(f'../data/{data_type}_info.parquet')
        np.save(f'../data/{data_type}_time.npy', time_array)
        np.save(f'../data/{data_type}_trial.npy', trial_array)

    return info, time_array, trial_array

In [29]:
left_real_info, left_real_times, left_real_trials = extract_data(data, 'left_real', save=True)

In [32]:
left_real_trials[:10]

array([[-1.03598653e-05, -1.98608647e-05, -2.76844564e-05, ...,
                    nan,             nan,             nan],
       [ 2.68916453e-06,  4.88606881e-06,  6.58706949e-06, ...,
                    nan,             nan,             nan],
       [ 4.45149519e-06,  2.49801333e-06,  1.00598444e-06, ...,
                    nan,             nan,             nan],
       ...,
       [ 9.58256683e-07,  1.94050894e-07, -4.05722838e-07, ...,
                    nan,             nan,             nan],
       [ 5.56583477e-06,  4.03205868e-06,  2.74496561e-06, ...,
                    nan,             nan,             nan],
       [ 5.80368395e-06,  3.65099512e-06,  1.65556952e-06, ...,
                    nan,             nan,             nan]])